<a href="https://colab.research.google.com/github/zhiyuan-95/portfolio_optimizer/blob/master/portfolio_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pandas_datareader import data as pdr
import yfinance as yfin
from datetime import datetime, timedelta,date
import time
import pandas as pd
import numpy as np

In [2]:
yfin.pdr_override()

In [10]:
class portfolio:
  def __init__(self,tickers):
    self._tickers = tickers
    self._div_history = pd.DataFrame()
    self._price_history = pd.DataFrame()
    self.get_stocks()
    self._t0 = ''
    self._period = 12
    self._t1 = ''
    self._price= pd.DataFrame()
    self._return = []
    self._div = pd.DataFrame()
  def get_stocks(self):
      file_names = []
      temp = []
      div_date = []
      div_history = {}
      iter = 0
      for x in self._tickers:
        Start = '2007-1-1'
        End = '2023-1-1'
        self._price_history[x] = pdr.get_data_yahoo(x,start=Start, end=End)['Adj Close']
        div_history[x] = yfin.Ticker(x).dividends
        div_date+=list(div_history[x].index)
      div_date = sorted(div_date)
      self._div_history.index= map(lambda x: x.strftime('%Y-%m-%d'), div_date)
      for y in self._tickers:
        self._div_history[y] =div_history[y]
        self._div_history[y] = self._div_history[y].fillna(0)
  def update(self, date):
    self._t1 = datetime.strptime(date,'%Y-%m-%d')
    self._t0 = self._t1-timedelta(weeks = self._period)
    start = self._t0.strftime('%Y-%m-%d')
    end = self._t1.strftime("%Y-%m-%d")
    last_year = str(int(end[:4])-1)
    self._price = self._price_history[start:end].to_numpy()
    self._div = self._div_history[self._div_history.index.str.startswith(last_year)].to_numpy()
  def Return(self):
    # return  = return on stock value from t0-t1 + expected on dividend
    # expected return on dividend of period t of year n = sum of dividend of year n-1 *(t/52)
    p0 = self._price[0]
    p1 = self._price[1]
    return_p = (p1-p0)/p0
    div_rate = sum(self._div)/p0
    E_return_d = div_rate*self._period/52
    return return_p+E_return_d
  def pastReturn(self,years=3):
    rtn = np.empty((3,years))
    for t in range(years):
      the_year = str(int(self._t1.strftime("%Y-%m-%d")[:4])-t-1)
      stock_last_yr= self._price_history[self._price_history.index.strftime("%Y-%m-%d").str.startswith(the_year)].to_numpy()
      div_sum = self._div_history[self._div_history.index.str.startswith(the_year)].to_numpy()
      rtn[t] = (sum(div_sum)+stock_last_yr[-1]-stock_last_yr[0])/stock_last_yr[0]
    return sum(rtn)/3

In [11]:
tickers = ['qqq','dog','spy']
p = portfolio(tickers)
p.update('2021-02-03')
p.pastReturn()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


array([ 0.28378857, -0.10545031,  0.16476169])

array([-0.00360184,  0.01079586, -0.00586434])